# Import

In [1]:
import csv
import itertools
from glob import glob

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from gensim.corpora import Dictionary
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm
import os
from nltk import word_tokenize
from nltk.corpus import stopwords

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Define functions

In [3]:
def get_corpus(docs):
    return [set(doc.split(' ')) for doc in docs]

In [4]:
def word_occurences(words, corpus):
    return sum([set(words)<=bag_of_words for bag_of_words in corpus])

In [5]:
def norm_pointwise_mutual_information(word_i, word_j, corpus):
    log_p_i = np.log(word_occurences([word_i], corpus)/ len(corpus))
    log_p_j = np.log(word_occurences([word_j], corpus)/len(corpus))
    log_p_ij = np.log(word_occurences([word_i, word_j], corpus)/len(corpus))
    return (log_p_i + log_p_j) / log_p_ij - 1

In [6]:
# def norm_pointwise_mutual_information(prob_i, prob_j, prob_ij):
#     log_p_i = np.log(prob_i)
#     log_p_j = np.log(prob_j)
#     log_p_ij = np.log(prob_ij)
#     return (log_p_i + log_p_j) / log_p_ij - 1

In [7]:
ag_news_df = pd.read_csv(
    "../data/CharCnn_Keras-master/data/ag_news_csv/train.csv",
    header=None,
    names=["class", "title", "description"],
)

In [8]:
ag_news_df["words"] = ag_news_df.description.progress_apply(word_tokenize)

100%|██████████| 120000/120000 [02:44<00:00, 730.69it/s]


In [11]:
ag_news_df.words = ag_news_df.words.progress_apply(lambda words: [word for word in words if word not in set(stopwords.words('english'))])

100%|██████████| 120000/120000 [2:57:04<00:00, 11.29it/s]  


In [ ]:
corpus = get_corpus(ag_news_df.description)

In [ ]:
# dictionary = Dictionary([set(doc.split(' ')) for doc in ag_news_df.description])

In [ ]:
count = 1

In [ ]:
dim_nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [ ]:
TC_dim = []
for dim_num in tqdm(dim_nums):
    with open(f"./data/gmm_pred/{dim_num}/{count}.csv", "r") as f:
        reader = csv.reader(f)
        preds = np.array([label for label in reader])
        
    preds = preds.reshape(-1)
    preds = np.array([int(label) for label in preds])
    
    ag_news_df[f"pred_{dim_num}_{count}"] = preds
    ag_news_df[f"pred_{dim_num}_{count}"] = ag_news_df[f"pred_{dim_num}_{count}"] + 1
    
    TC = 0
    for label in range(1, 5):
        df_class = ag_news_df[ag_news_df[f"pred_{dim_num}_{count}"] == label]
        dictionary_class = Dictionary([set(doc.split(" ")) for doc in df_class.description])
        common_words = [word for word, count in dictionary_class.most_common(n=10)]
        
        os.makedirs(f"./data/gmm_pred_commonword/{dim_num}/", exist_ok=True)
        with open(f"./data/gmm_pred_commonword/{dim_num}/{count}.csv", "a") as f:
            writer = csv.writer(f)
            writer.writerow(common_words)


        TC_class = 0
        for word_i, word_j in itertools.combinations(common_words, 2):
            TC_class = TC_class + norm_pointwise_mutual_information(word_i, word_j, corpus)
        TC_class = TC_class / len(list(itertools.combinations(common_words, 2)))

        TC = TC + TC_class
    TC = TC / len(range(1, 5))
    TC_dim.append(TC)
with open(f"./data/TC.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(TC_dim)

 13%|█▎        | 2/15 [04:07<26:43, 123.38s/it]

In [ ]:
ag_news_df.words

In [ ]:
TC_dim_nonstop = []
for dim_num in tqdm(dim_nums):
    with open(f"./data/gmm_pred/{dim_num}/{count}.csv", "r") as f:
        reader = csv.reader(f)
        preds = np.array([label for label in reader])
        
    preds = preds.reshape(-1)
    preds = np.array([int(label) for label in preds])
    
    ag_news_df[f"pred_{dim_num}_{count}"] = preds
    ag_news_df[f"pred_{dim_num}_{count}"] = ag_news_df[f"pred_{dim_num}_{count}"] + 1
    
    TC = 0
    for label in range(1, 5):
        df_class = ag_news_df[ag_news_df[f"pred_{dim_num}_{count}"] == label]
        dictionary_class = Dictionary(df_class.words)
        
        common_words = [word for word, count in dictionary_class.most_common(n=10)]
        
        os.makedirs(f"./data/gmm_pred_commonword_nonstop/{dim_num}/", exist_ok=True)
        with open(f"./data/gmm_pred_commonword_nonstop/{dim_num}/{count}.csv", "a") as f:
            writer = csv.writer(f)
            writer.writerow(common_words)

        TC_class = 0
        for word_i, word_j in itertools.combinations(common_words, 2):
            TC_class = TC_class + norm_pointwise_mutual_information(word_i, word_j, corpus)
        TC_class = TC_class / len(list(itertools.combinations(common_words, 2)))

        TC = TC + TC_class
    TC = TC / len(range(1, 5))
    TC_dim_nonstop.append(TC)
with open(f"./data/TC_nonstop.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(TC_dim_nonstop)

In [62]:
df_class = ag_news_df[ag_news_df[f"pred_{dim_num}_{count}"]==label]

In [71]:
dictionary_class = Dictionary([set(doc.split(' ')) for doc in df_class.description])

In [72]:
common_words = [word for word, count in dictionary_class.most_common(n=10)]

In [73]:
common_words

['the', 'to', 'of', 'a', 'and', 'in', 'on', '-', 'for', '']

In [86]:
TC = 0
for word_i, word_j in tqdm(itertools.combinations(common_words, 2)):
    TC = TC + norm_pointwise_mutual_information(word_i, word_j, corpus)
TC = TC / len(list(itertools.combinations(common_words, 2)))

45it [00:25,  1.74it/s]


In [87]:
TC

0.024271094345515165

In [7]:
dim_num =2
count = 1

In [37]:
with open(f"./data/gmm_pred/{dim_num}/{count}.csv", "r") as f:
    reader = csv.reader(f)
    preds = np.array([label for label in reader])

In [38]:
preds = preds.reshape(-1)
preds = np.array([int(label) for label in preds])

In [89]:
ag_news_df[f"pred_{dim_num}_{count}"] = preds
ag_news_df[f"pred_{dim_num}_{count}"] = ag_news_df[f"pred_{dim_num}_{count}"] + 1

In [91]:
TC = 0
for label in range(1, 5):
    df_class = ag_news_df[ag_news_df[f"pred_{dim_num}_{count}"] == label]
    dictionary_class = Dictionary([set(doc.split(" ")) for doc in df_class.description])
    common_words = [word for word, count in dictionary_class.most_common(n=10)]

    TC_class = 0
    for word_i, word_j in tqdm(itertools.combinations(common_words, 2)):
        TC_class = TC_class + norm_pointwise_mutual_information(word_i, word_j, corpus)
    TC_class = TC_class / len(list(itertools.combinations(common_words, 2)))
    
    TC = TC + TC_class
TC = TC / len(range(1, 5))

45it [00:24,  1.87it/s]
45it [00:24,  1.84it/s]
45it [00:24,  1.82it/s]
45it [00:23,  1.91it/s]
